In [1]:
!pip install -U ../input/kerasapplications/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/qubvel/efficientnet-1.0.0-py3-none-any.whl
!pip install ../input/qubvel/image_classifiers-1.0.0-py3-none-any.whl
!pip install ../input/qubvel/segmentation_models-1.0.0-py3-none-any.whl

Processing /kaggle/input/kerasapplications/Keras_Applications-1.0.8-py3-none-any.whl
Processing /kaggle/input/qubvel/efficientnet-1.0.0-py3-none-any.whl
Processing /kaggle/input/qubvel/image_classifiers-1.0.0-py3-none-any.whl
Processing /kaggle/input/qubvel/segmentation_models-1.0.0-py3-none-any.whl


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import tifffile as tiff
from tqdm.auto import tqdm
from glob import glob
import gc
import os
import warnings 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
gc.enable()
warnings.simplefilter('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
import segmentation_models as sm
#from efficientnet.tfkeras import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7 

import albumentations as A

Segmentation Models: using `keras` framework.


In [3]:
cfg = {
    'train_df': '../input/hubmap-organ-segmentation/train.csv',
    'train_img': '../input/hubmap-organ-segmentation/train_images/',
    'train_annot': '../input/hubmap-organ-segmentation/train_annotations/',
    'img_shape': 384,
    'thresh': 0.3
}

In [4]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou


def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(tf.cast(y_true, tf.float32), y_pred) + 0.5 * dice_loss(tf.cast(y_true, tf.float32), y_pred)

def weighted_loss(y_true, y_pred):
    # Calculate the base loss
    ce = K.sparse_categorical_crossentropy(y_true, y_pred)
    # Apply the weights
    one_weight = 1.0
    zero_weight = 1e-2
    weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
    weight_vector = K.squeeze(weight_vector, axis=-1)
    weighted_ce = weight_vector * ce

    # Return the mean error
    return K.mean(weighted_ce)

In [5]:
def mask2rle(mask, orig_dim=160):
    #Rescale image to original size
    size = int(len(mask.flatten())**.5)
    n = Image.fromarray(mask.reshape((size, size))*255.0)
    n = n.resize((orig_dim, orig_dim))
    n = np.array(n).astype(np.float32)
    #Get pixels to flatten
    pixels = n.T.flatten()
    #Round the pixels using the half of the range of pixel value
    pixels = (pixels-min(pixels) > ((max(pixels)-min(pixels))/2)).astype(int)
    pixels = np.nan_to_num(pixels) #incase of zero-div-error
    
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0]
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

In [6]:
# model = tf.keras.models.load_model('../input/deeplabv3-bc-logits-false/best_model.h5', custom_objects={'dice_coef': dice_coef, 'iou_coef': iou_coef, 'dice_loss': dice_loss, 'bce_dice_loss': bce_dice_loss, 'weighted_loss': weighted_loss}, compile=False)
model = tf.keras.models.load_model('../input/fpn-effnetb7-384p/best_model.h5', custom_objects={'binary_crossentropy_plus_jaccard_loss': sm.losses.bce_jaccard_loss, 'dice_coef': dice_coef, 'iou_coef': iou_coef, 'dice_loss': dice_loss, 'f1-score': sm.metrics.f1_score, 'binary_crossentropy_plus_dice_loss':sm.losses.bce_dice_loss})
test_df = pd.read_csv('../input/hubmap-organ-segmentation/test.csv')
sub = {'id':[], 'rle':[]}

for _, row in tqdm(test_df.iterrows()):
    path = '../input/hubmap-organ-segmentation/test_images/' + str(row['id']) + '.tiff'
    img = Image.open(path)
    img = img.resize((cfg['img_shape'], cfg['img_shape']))
    img = np.array(img).reshape((1, cfg['img_shape'], cfg['img_shape'], 3))
    img = img / 255.0
    mask = model.predict(img)
    #mask = np.where(mask >= cfg['thresh'], 1, 0)
    
    rle = mask2rle(np.round(mask).astype(np.float32), row['img_width']) 
    sub['id'].append(row['id'])
    sub['rle'].append(rle)
    
    
sub = pd.DataFrame(sub)
sub.to_csv('submission.csv', index=False)
sub

0it [00:00, ?it/s]

,id,rle
0,10078,21803 4 21861 5 23824 66 25846 68 27869 69 298...
